# ETL Pipeline - Import Libraries

This is the first step of the ETL (Extract, Transform, Load) pipeline.

We import essential Python libraries required for data extraction, transformation, and loading:

- **pandas**: for reading and manipulating structured data
- **numpy**: for numerical operations and handling missing values

This library will support the data cleaning and preparation tasks for downstream analysis including:
- Customer Behavior Analysis
- Product Trend Analysis
- Sales Trend Analysis

In [9]:
import pandas as pd
import numpy as np


# Step 1: Extract


## Load Dataset with Encoding(Extract)

The Online Retail dataset contains special characters such as the British pound symbol (£) and accented letters (é, ü, etc.).

To ensure these characters are read correctly and to avoid `UnicodeDecodeError`, we specify the encoding format as `ISO-8859-1` (also known as Latin-1), which is commonly used for Western European text.

This step loads the raw dataset into a pandas DataFrame for further processing.



In [10]:
df = pd.read_csv('../data/raw/Online Retail.csv', encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


#  Step 2: Transfer

##  Initial Data Exploration

In this step, we perform an initial exploration of the dataset to understand its structure and contents.

- `df.shape` shows the number of rows and columns.
- `df.info()` provides a summary of data types and missing values.
- `df.describe(include='all')` gives statistical summaries for both numerical and categorical columns.

This helps identify potential data quality issues and guides the next steps in the cleaning and transformation process.


In [11]:
print("Initial shape:", df.shape)
df.info()
df.describe(include='all')

Initial shape: (541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   541909 non-null  int64  
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 33.1+ MB


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909.000000,541909,541909.000000,541909.000000,541909
unique,25900,4070,4223,NaN,23260,NaN,NaN,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-10-31 14:41:00,NaN,NaN,United Kingdom
freq,1114,2313,2369,NaN,1114,NaN,NaN,495478
mean,NaN,NaN,NaN,9.552250,NaN,4.611114,15287.518434,NaN
std,NaN,NaN,NaN,218.081158,NaN,96.759853,1484.746041,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,14367.000000,NaN
50%,NaN,NaN,NaN,3.000000,NaN,2.080000,15287.000000,NaN
75%,NaN,NaN,NaN,10.000000,NaN,4.130000,16255.000000,NaN


##  Handle Missing Values

In this step, we remove rows where either `CustomerID` or `Description` is missing.

These columns are important for our project because:

- `CustomerID` is needed to analyze customer behavior.
- `Description` is needed to understand which products were purchased.

Since we are not analyzing anonymous customers or incomplete product records at this stage, we don't need these rows for our analysis. Dropping them helps keep the data clean and ensures accurate insights for:

1. Customer behavior analysis  
2. Product trend analysis  
3. Sales trend analysis


In [12]:
df = df.dropna(subset=['CustomerID', 'Description'])
print("After dropping missing CustomerID/Description:", df.shape)


After dropping missing CustomerID/Description: (540455, 8)


##  Remove Cancelled Orders

Invoices starting with the letter 'C' indicate **cancelled transactions** or returns. This is a common convention in retail datasets, where 'C' stands for **Cancelled** or **Credit**.

We remove these cancelled orders because:

- They represent **refunds or reversed sales**, not actual purchases.
- Including them would **distort customer behavior analysis**, such as purchase frequency or spending.
- They would also **mislead product trend analysis** by showing false product sales or popularity.
- Sales trends would be less accurate if cancellations are included.

Dropping cancelled orders helps keep the analysis focused on **real sales and customer activity**.

If you want to analyze returns or cancellations separately, you can keep these rows in a different dataset for further study.

Overall, removing cancelled orders **improves the accuracy** of our customer behavior, product trends, and sales trend analyses.



In [6]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
print("After removing cancellations:", df.shape)


After removing cancellations: (531167, 8)


## Remove Non-Positive Values

We remove rows where `Quantity` or `UnitPrice` are zero or negative.

- A **positive Quantity** means products were actually bought.
- A **positive UnitPrice** means the item had a valid price.

Rows with zero or negative values could represent errors, free samples, or returns already handled elsewhere.

Removing them ensures our analysis reflects **real sales transactions**.



In [13]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
print("After removing zero/negative Quantity and UnitPrice:", df.shape)



After removing zero/negative Quantity and UnitPrice: (530104, 8)


## Convert `InvoiceDate` from Object to DateTime



We convert it to datetime type to enable time-based analysis, such as grouping by year, month, day, or hour.

This conversion allows easier manipulation and calculation of date-related features.



In [14]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [21]:
print(df['InvoiceDate'].head())
print(df['InvoiceDate'].dtype)


0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
3   2010-12-01 08:26:00
4   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]
datetime64[ns]


In [15]:
# Create new columns
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['Hour'] = df['InvoiceDate'].dt.hour
df['Weekday'] = df['InvoiceDate'].dt.day_name()



In [16]:
#to check the new columns
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'Year', 'Month',
       'Day', 'Hour', 'Weekday'],
      dtype='object')


## Rename Description Column and change it to lowercase

We rename the `Description` column to `ProductName` for better clarity.

Then, we clean the `ProductName` column by:

- Removing extra spaces at the start and end (`strip()`).
- Converting all text to lowercase (`lower()`).

This standardizes product names, helping to avoid duplicates due to differences in case or spacing and improving product trend analysis.



In [17]:
df = df.rename(columns={'Description': 'ProductName'})
df['ProductName'] = df['ProductName'].str.strip().str.lower()
df.head


<bound method NDFrame.head of        InvoiceNo StockCode                          ProductName  Quantity  \
0         536365    85123A   white hanging heart t-light holder         6   
1         536365     71053                  white metal lantern         6   
2         536365    84406B       cream cupid hearts coat hanger         8   
3         536365    84029G  knitted union flag hot water bottle         6   
4         536365    84029E       red woolly hottie white heart.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          pack of 20 spaceboy napkins        12   
541905    581587     22899          children's apron dolly girl         6   
541906    581587     23254         childrens cutlery dolly girl         4   
541907    581587     23255      childrens cutlery circus parade         4   
541908    581587     22138         baking set 9 piece retrospot         3   

               InvoiceDate  UnitPrice  Custom

In [18]:
#Remove duplicates
df = df.drop_duplicates()
print("After removing duplicates:", df.shape)


After removing duplicates: (524878, 14)


###  Dataset Shape Before and After Cleaning

- **Initial shape:** (541,909 rows, 8 columns)  
- **After cleaning:** (524,878 rows, 14 columns)  

Cleaning reduced the number of rows by removing missing values, cancellations, non-positive quantities/prices, and duplicates.

The number of columns increased because we created new features like `TotalPrice`, `Year`, `Month`, `Day`, `Hour`, and `Weekday` to support deeper analysis.


# Step 3: Load




In [19]:
#Save cleaned data

df.to_csv('cleaned_online_retail.csv', index=False)
print("Cleaned dataset saved as 'cleaned_online_retail.csv'")

Cleaned dataset saved as 'cleaned_online_retail.csv'


In [ ]:
#to check the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 524878 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    524878 non-null  object        
 1   StockCode    524878 non-null  object        
 2   ProductName  524878 non-null  object        
 3   Quantity     524878 non-null  int64         
 4   InvoiceDate  524878 non-null  datetime64[ns]
 5   UnitPrice    524878 non-null  float64       
 6   CustomerID   524878 non-null  int64         
 7   Country      524878 non-null  object        
 8   TotalPrice   524878 non-null  float64       
 9   Year         524878 non-null  int32         
 10  Month        524878 non-null  int32         
 11  Day          524878 non-null  int32         
 12  Hour         524878 non-null  int32         
 13  Weekday      524878 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(4), int64(2), object(5)
memory usage: 52.1+ MB
